In [1]:
%pip install embeddingstore --extra-index-url https://azuremlsdktestpypi.azureedge.net/embeddingstore/

Looking in indexes: https://pypi.org/simple, https://azuremlsdktestpypi.azureedge.net/embeddingstore/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.4/84.4 kB 225.6 kB/s eta 0:00:00a 0:00:01
  Using cached Flask-2.3.2-py3-none-any.whl (96 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 1.1 MB/s eta 0:00:00a 0:00:01
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached itsdangerous-2.1.2-py3-none-any.whl (15 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 4.8 MB/s eta 0:00:00
  Using cached blinker-1.6.2-py3-none-any.whl (13 kB)
  Using cached MarkupSafe-2.1.3-cp310-cp310-macosx_10_9_universal2.whl (17 kB)

[notice] A new release of pip is available: 23.1.2 -> 23.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### **Load Environment variables from .env file**

In [1]:
import os
from typing import List
from dotenv import load_dotenv
import urllib.request
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter
import openai
from embeddingstore.core.contracts import (
    EmbeddingModelType,
    StorageType,
    StoreCoreConfig,
)
from embeddingstore.core.embeddingstore_core import EmbeddingStoreCore

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") 
OPENAI_DEPLOYMENT_ENDPOINT = os.getenv("OPENAI_DEPLOYMENT_ENDPOINT")
OPENAI_DEPLOYMENT_NAME = os.getenv("OPENAI_DEPLOYMENT_NAME")
OPENAI_MODEL_NAME = os.getenv("OPENAI_MODEL_NAME")
OPENAI_DEPLOYMENT_VERSION = os.getenv("OPENAI_DEPLOYMENT_VERSION")

OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = os.getenv("OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME")
OPENAI_ADA_EMBEDDING_MODEL_NAME = os.getenv("OPENAI_ADA_EMBEDDING_MODEL_NAME")

# Configure OpenAI API
openai.api_type = "azure"
openai.api_version = OPENAI_DEPLOYMENT_VERSION
openai.api_base = OPENAI_DEPLOYMENT_ENDPOINT
openai.api_key = OPENAI_API_KEY

In [2]:
URL_PREFIX = "https://learn.microsoft.com/en-us/azure/machine-learning/"
URL_NAME_LIST = [
    "overview-what-is-azure-machine-learning",
]

### **Download the data**

In [3]:
local_file_path = os.path.join(os.getcwd(), "data")
print(local_file_path)
os.makedirs(local_file_path, exist_ok=True)
for url_name in URL_NAME_LIST:
    url = os.path.join(URL_PREFIX, url_name)
    destination_path = os.path.join(local_file_path, url_name)
    urllib.request.urlretrieve(url, destination_path)

/Users/vladfeigin/myprojects/openai/workshops/dataai/openaiworkshop-new/openaiworkshop/Lab #3 - Create embeddings in pandas/data


#### **Create simple vector store**

In [4]:
DIMENSION = 1536

# Configure an embedding store to store index file.
store_path = os.path.join(os.getcwd(), "faiss_index_store")
config = StoreCoreConfig.create_config(
    storage_type=StorageType.LOCAL,
    store_identifier=store_path,
    model_type=EmbeddingModelType.AOAI,
    model_api_base=OPENAI_DEPLOYMENT_ENDPOINT,
    model_api_key=OPENAI_API_KEY,
    model_api_version=OPENAI_DEPLOYMENT_VERSION,
    model_name=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME,
    dimension=DIMENSION,
    create_if_not_exists=True,
)
store = EmbeddingStoreCore(config)

In [5]:
help(EmbeddingStoreCore)

Help on class EmbeddingStoreCore in module embeddingstore.core.embeddingstore_core:

class EmbeddingStoreCore(builtins.object)
 |  EmbeddingStoreCore(config: embeddingstore.core.contracts.config.StoreCoreConfig)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, config: embeddingstore.core.contracts.config.StoreCoreConfig)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  batch_insert_texts(self, texts: Iterable[str], metadatas: Optional[List[dict]] = None) -> None
 |  
 |  batch_insert_texts_with_embeddings(self, texts: Iterable[str], embeddings: Iterable[List[float]], metadatas: Optional[List[dict]] = None) -> None
 |  
 |  clear(self)
 |  
 |  save(self)
 |  
 |  search_by_embedding(self, query_embedding: List[float], top_k: int = 5) -> List[embeddingstore.core.contracts.entities.SearchResultEntity]
 |  
 |  search_by_text(self, query_text: str, top_k: int = 5) -> List[embeddingstore.core.contracts.entities.SearchResultEntity]
 |  
 |  -------------

In [5]:
def get_file_chunks(file_name: str) -> List[str]:
    with open(file_name, "r", encoding="utf-8") as f:
        page_content = f.read()
        # use BeautifulSoup to parse HTML content
        soup = BeautifulSoup(page_content, "html.parser")
        text = soup.get_text(" ", strip=True)
        chunks = []
        splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=30)
        for chunk in splitter.split_text(text):
            chunks.append(chunk)
        return chunks

### **Chunk every file and save in a vector store**

In [6]:
for root, _, files in os.walk(local_file_path):
    for file in files:
        each_file_path = os.path.join(root, file)

        # Split the file into chunks.
        chunks = get_file_chunks(each_file_path)
        count = len(chunks)
        if URL_PREFIX is not None:
            metadatas = [
                {"title": file, "source": os.path.join(URL_PREFIX, file)}
            ] * count
        else:
            metadatas = [{"title": file}] * count

        # Embed chunks into embeddings, generate index in embedding store.
        # If your data is large, inserting too many chunks at once may cause
        # rate limit error，you can refer to the following link to find solution
        # https://learn.microsoft.com/en-us/azure/cognitive-services/openai/quotas-limits
        store.batch_insert_texts(chunks, metadatas)
        print(f"Create index for {file} file successfully.\n")

Create index for recipes_onecol_with_embeddings.csv file successfully.

Create index for recipes_onecol.csv file successfully.



FileNotFoundError: [Errno 2] No such file or directory: '/Users/vladfeigin/myprojects/openai/workshops/dataai/openaiworkshop-new/openaiworkshop/Lab #3 - Create embeddings in pandas/data/overview-what-is-azure-machine-learning'

In [ ]:
result = store.search_by_text("Azure Machine Learning", top_k=5)